<a href="https://colab.research.google.com/github/MattiaPOLI/DR14/blob/master/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

from google.colab import files
uploaded = files.upload()

MessageError: ignored

In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import IPython
import plotly
import plotly.graph_objs as go
import plotly.plotly as py
import pandas as pd
import numpy as np
import io

def enable_plotly_in_cell():
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  plotly.offline.init_notebook_mode(connected=True)

df = pd.read_csv(io.StringIO(uploaded['Sky.csv'].decode('utf-8')))
#store the labels into a different array, we must not use them during PCA
labels = np.array(df.iloc[:,13])
classes = np.unique(df['class'].values).tolist()
class_code = {classes[k]: k for k in range(3)}
color_vals = [class_code[cl] for cl in df['class']]
#dropping columns related to the struments used for retrieving measures 
#run/rerun/camcol/field are parameters used to identify from which scan data are taken. Scans in database are accessed with the hash specobjd
#plate/mjd/fiberid are parameters related to the strumentation used
#both are probably going to spoil the result because actually not related to the celestial object
df2 = df.drop(columns=["objid", "run", "rerun", "camcol", "field", "specobjid", "plate", "mjd", "fiberid", "class"])

df2.dtypes

dfStandard = StandardScaler().fit_transform(df2)
PCA = PCA()
data_PCA = PCA.fit_transform(dfStandard)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import cross_val_score

neighbors = [1, 5, 10, 25]
p = [1, 2, 3, 4]
scores = np.zeros(shape = (len(neighbors), len(p)))

fifthPCA = data_PCA[:, 0:5]

trainSet, test, labelTrainSet, labelTest = train_test_split(fifthPCA, labels, train_size = 0.7, test_size = 0.3, random_state = 40)
for i in range(len(neighbors)):
  for j in range(len(p)):
    clf = KNN(n_neighbors = neighbors[i], p = p[j], metric = "minkowski")
    scores[i, j] = cross_val_score(clf, trainSet, labelTrainSet, cv = 10).mean()
    
    
trace = go.Heatmap(
    z = scores,
    colorscale = "Blue"
)
layout = go.Layout(
    xaxis = dict(
        tickmode = "array",
        tickvals = (0,1,2,3),
        ticktext = p,
        title = "P"        
    ),
    yaxis = dict(
        tickmode = "array",
        tickvals = (0,1,2,3),
        ticktext = neighbors,
        title = "Neighbors"
    )
)
data = [trace]
figure = go.Figure(data = data, layout = layout)
plotly.offline.iplot(figure, filename="KNNAccuracyK-fold") 